In [1]:
import pandas as pd
from sklearn.preprocessing import label_binarize
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from keras.utils import np_utils
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import statistics



Using TensorFlow backend.


In [3]:
with open("Result_Folder/Amp_ISS.csv",'w') as resultfile:
    header='user_no,accu,happy_accu,sad_accu,stress_accu,relax_accu\n'
    resultfile.write(header)
    user_dir="SMOTE_folder/Amp_ISS/"
    
    for user in range(1,23,1):
            filename=user_dir+"U"+str(user)+"_LIHF_itd_file.csv" 
            #filename=user_dir+"user_"+str(user)+"_v4.csv"
            print(filename)
            dataset = pd.read_csv(filename)
            array = dataset.values
            np.random.shuffle(array)
            col_no=array.shape[1]-1
            raw_col=8
            #fea_no=15
            print("col_no="+str(col_no))
            X_set=array[:,0:col_no]
            #X_set=array[:,0:fea_no]
            #binarize the label
            #Y_set=label_binarize(array[:,col_no],classes=['Happy','Sad','Stressed','Relaxed'])
            Y_set=label_binarize(array[:,col_no],classes=[2.0,-2.0,1.0,0.0])
            n_classes=Y_set.shape[1]
            #print(Y_set)
            kfold = KFold(10, True, 1)
            
            data_array=np.concatenate((X_set,Y_set),axis=1)
            #print(data_array.shape)
            roc_auc_avg=[]
            happy_list=[]
            sad_list=[]
            stressed_list=[]
            relaxed_list=[]
            dummy_col=col_no+4
            #dummy_col=fea_no+4
            print("dummy_col="+str(dummy_col))
            
            for train, test in kfold.split(data_array):
                #print('train: %s, test: %s' % (len(data_array[train]), len(data_array[test])))
                train_set=data_array[train]
                test_set=data_array[test]
                X_train=train_set[:,0:col_no]
                Y_train=train_set[:,col_no:dummy_col]
                Y_train=Y_train.astype('int')
                X_test=test_set[:,0:col_no]
                Y_test=test_set[:,col_no:dummy_col]
                Y_test=Y_test.astype('int')
                #print(Y_test.shape)
                fpr = dict()
                tpr = dict()
                roc_auc = dict()
                f_score=dict()
                #Learn to predict each class against the other
                num_classes=4
                batch_size = 10
                epochs = 50
                
    
                model = Sequential()
                model.add(Dense(10, input_dim=10, activation='relu'))
                #model.add(Dropout(0.3))
                #model.add(Dense(14, activation='tanh'))
                #model.add(Dropout(0.2))
                model.add(Dense(15, activation='relu'))
                model.add(Dense(num_classes, activation='softmax'))
                model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
                history = model.fit(X_train,Y_train,batch_size=batch_size,verbose=0, epochs=epochs,
                        validation_data=(X_test, Y_test))
                Y_prob = model.predict(X_test)
            
                print("Y_prob.shape[0]="+str(Y_prob.shape[0]))
                
                for i in range(0,Y_prob.shape[0]):
                    max_idx=np.argmax(Y_prob[i,:])
                    for j in range(0,num_classes):
                        if j==max_idx:
                            Y_prob[i,j]=1
                        else:
                            Y_prob[i,j]=0
                
                
                
                
                #clf=OneVsRestClassifier(RandomForestClassifier(n_estimators=100))
                #Y_prob=clf.fit(X_train,Y_train).predict_proba(X_test)
                #Y_pred=clf.fit(X_train,Y_train).predict(X_test)
                #print(Y_pred)
                #compute AUCROC for each class
                for i in range(n_classes):
                    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], Y_prob[:, i])
                    roc_auc[i] = auc(fpr[i], tpr[i]) 
                    
                    
                    if(math.isnan(roc_auc[i])):
                        continue
                    elif(i==0):
                        happy_list.append(roc_auc[i])
                        
                    elif(i==1):
                        sad_list.append(roc_auc[i])
                        
                    elif(i==2):
                        stressed_list.append(roc_auc[i])
                        
                    else:
                        relaxed_list.append(roc_auc[i])
                        
                #print(roc_auc) 
                try:    
                    weighted_roc_auc_ovr = roc_auc_score(Y_test, Y_prob,multi_class="ovr",average="weighted")
                    #print(weighted_roc_auc_ovr)
                    roc_auc_avg.append(weighted_roc_auc_ovr)
                except ValueError:
                    pass

            if(len(roc_auc_avg)!=0):    
                print("weighted roc score using method=",statistics.mean(roc_auc_avg))
                if(len(roc_auc_avg)>1):
                    print("standard deviation of roc score="+str(statistics.stdev(roc_auc_avg)))
                    
            #for auc-roc    
            happy_score,sad_score,stress_score,relax_score=0,0,0,0
            if(len(happy_list)!=0):
                happy_score=statistics.mean(happy_list)
                print("happy score=",happy_score)
                if(len(happy_list)>1):
                    print("happy standard deviation="+str(statistics.stdev(happy_list)))
            if(len(sad_list)!=0):
                sad_score=statistics.mean(sad_list)
                print("sad_score=",sad_score) 
                if(len(sad_list)>1):
                    print("sad standard deviation="+str(statistics.stdev(sad_list)))
            if(len(stressed_list)!=0):
                stress_score=statistics.mean(stressed_list)
                print("stressed_score=",stress_score)
                if(len(stressed_list)>1):
                    print("stress standard deviation="+str(statistics.stdev(stressed_list)))
            if(len(relaxed_list)!=0):    
                relax_score=statistics.mean(relaxed_list)
                print("relax_score=",relax_score) 
                if(len(relaxed_list)>1):
                    print("relax standard deviation="+str(statistics.stdev(relaxed_list)))
                
    
            roc_auc_weight=0

            for i in range(n_classes):
                no_entry=sum(Y_set[:,i]==1)
                if(i==0):
                    happy_entry=no_entry
                elif(i==1):
                     sad_entry=no_entry
                elif(i==2):
                    stress_entry=no_entry
                else:
                     relax_entry=no_entry
            roc_auc_weight=((happy_entry*happy_score)+(sad_entry*sad_score)+(stress_entry*stress_score)+(relax_score*relax_entry))/X_set.shape[0]    
            print("weighted roc score using formula=",roc_auc_weight)  
            line=filename+','+str(roc_auc_weight)+','+str(happy_score)+','+str(sad_score)+','+str(stress_score)+','+str(relax_score)+'\n'
            resultfile.write(line)

SMOTE_folder/Amp_ISS/U1_LIHF_itd_file.csv
col_no=10
dummy_col=14
Y_prob.shape[0]=14
Y_prob.shape[0]=13
Y_prob.shape[0]=13


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


Y_prob.shape[0]=13
Y_prob.shape[0]=13


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


Y_prob.shape[0]=13
Y_prob.shape[0]=13
Y_prob.shape[0]=13
Y_prob.shape[0]=13
Y_prob.shape[0]=13
weighted roc score using method= 0.5055555555555555
standard deviation of roc score=0.03412583054528701
happy score= 0.48930555555555555
happy standard deviation=0.0413467657882967
sad_score= 0.5740740740740741
sad standard deviation=0.1689656258416172
stressed_score= 0.5
stress standard deviation=0.0
relax_score= 0.5213293650793651
relax standard deviation=0.060495323512455304
weighted roc score using formula= 0.5188600609879236
SMOTE_folder/Amp_ISS/U2_LIHF_itd_file.csv
col_no=10
dummy_col=14
Y_prob.shape[0]=5


...

/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


Y_prob.shape[0]=5


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)


Y_prob.shape[0]=5
happy score= 0.5
happy standard deviation=0.0
stressed_score= 0.5
stress standard deviation=0.0
relax_score= 0.5
relax standard deviation=0.0
weighted roc score using formula= 0.5


/home/salma/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:808: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning)
